In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
boaring_words_base = spark.sparkContext.textFile("/user/itv017244/TT/boringwords.txt")

In [4]:
students_review_base = spark.sparkContext.textFile("/public/trendytech/reviews/trendytech-student-reviews.csv")

In [5]:
rdd1 = students_review_base.flatMap(lambda x:x.split(" ")).map(lambda x:x.lower())

In [6]:
rdd2 = rdd1.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)

In [7]:
rdd2.take(10)

[('was', 61),
 ('learning', 31),
 ('journey', 6),
 ('sumit', 109),
 ('i', 215),
 ('new', 5),
 ('technology', 3),
 ('them', 1),
 ('career', 28),
 ('of', 182)]

In [8]:
broadcast_bw = spark.sparkContext.broadcast(boaring_words_base.collect())

In [9]:
rdd3 = rdd2.filter(lambda x : x[0] not in broadcast_bw.value)

In [10]:
rdd4 = rdd3.reduceByKey(lambda x,y : x+y).sortBy(lambda x: x[1], ascending=False)

In [11]:
rdd4.take(20)

[('data', 201),
 ('sumit', 109),
 ('trendytech', 67),
 ('', 64),
 ('data.', 34),
 ('course.', 33),
 ("sir's", 23),
 ('trendy', 14),
 ('course,', 13),
 ("master's", 13),
 ('domain.', 12),
 ("trendytech's", 12),
 ('sir.', 11),
 ('program.', 9),
 ('field.', 9),
 ('concepts.', 9),
 ('hands-on', 8),
 ('fresher', 8),
 ('amazing.', 8),
 ('career.', 7)]